In [2]:
import pandas as pd

In [10]:
pokemon_151 = pd.read_csv('Pokemon_151.csv', index_col = 'Unnamed: 0')
pokemon_151

,Product,Date Range,Market Price
0,Code Card - 151 Booster Pack,7/15 to 7/17,$0.00
1,Code Card - 151 Booster Pack,7/18 to 7/20,$0.00
2,Code Card - 151 Booster Pack,7/21 to 7/23,$0.00
3,Code Card - 151 Booster Pack,7/24 to 7/26,$0.00
4,Code Card - 151 Booster Pack,7/27 to 7/29,$0.00
...,...,...,...
6895,151 Booster Bundle Display Case,9/28 to 9/30,$0.13
6896,151 Booster Bundle Display Case,10/1 to 10/3,$0.11
6897,151 Booster Bundle Display Case,10/4 to 10/6,$0.13
6898,151 Booster Bundle Display Case,10/7 to 10/9,$0.10


In [12]:
len(pokemon_151['Product'].unique())

229